2021 ~ 2023년  
김태균 교수님 피드백대도 다달이 10%의 질문을 샘플링 해서 해보자

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pickle
import re
from umap import UMAP
from hdbscan import HDBSCAN
import torch
from transformers import RobertaTokenizer, RobertaConfig, RobertaModel
from collections.abc import Iterable
from transformers import AutoTokenizer, AutoModel
from bertopic import BERTopic
import torch
from collections import deque
from bertopic.representation import KeyBERTInspired
from tqdm import tqdm
from sklearn.cluster import KMeans
import calendar
from bertopic.vectorizers import ClassTfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import openai
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, OpenAI, PartOfSpeech
from itertools import product

In [3]:
# sample_df
# 전체 데이터셋 
with open('../../data/sample_df.pkl', 'rb') as f:
    sample_df = pickle.load(f)

In [4]:
# 파일 불러오기
with open('../../data/bert_src_df.pkl', 'rb') as f:
    bert_src_df = pickle.load(f)


In [5]:
bert_src_df.info()
# 74308951

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4119261 entries, 0 to 4119260
Data columns (total 5 columns):
 #   Column          Dtype         
---  ------          -----         
 0   q_id            int64         
 1   a_id            float64       
 2   q_creationdate  datetime64[ns]
 3   tags            object        
 4   body            object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 157.1+ MB


In [6]:
bert_src_df[bert_src_df['q_id'] ==74308951]

,q_id,a_id,q_creationdate,tags,body
3697928,74308951,74309073.0,2022-11-03 20:03:18.853,<python><string><split><delimiter>,<p>I am trying to create a function to parse a...


In [7]:
cond1 = bert_src_df['q_creationdate']<='2022-11-30'
cond2 = bert_src_df['q_creationdate']>='2021-11-30'
cond3 = bert_src_df['a_id'].isna()
cond4 = bert_src_df['tags'].str.contains('python')
cond5 = bert_src_df['q_creationdate']>='2022-12-01'

In [8]:
bert_src = bert_src_df.loc[cond2 & cond4, :]

In [9]:
bert_src.head()

,q_id,a_id,q_creationdate,tags,body
1100212,70162722,70162757.0,2021-11-30 00:00:11.787,<python><python-3.x><unit-testing>,<p>I have this function:</p>\n<pre><code>def t...
1100217,70162741,70163172.0,2021-11-30 00:03:41.067,<python><pandas><dataframe><group-by><data-ana...,<p>I have a df with the following structure</p...
1100225,70162770,70162797.0,2021-11-30 00:07:54.443,<python><string><find><getvalue>,<p>I need to make an API that gets text from a...
1100232,70162795,70162823.0,2021-11-30 00:11:37.677,<python><pandas><dataframe>,<p>I have a dataframe <code>data</code> which ...
1100238,70162816,NaN,2021-11-30 00:15:44.530,<python><pandas><csv>,<p>I have a data frame that I'm trying to writ...


In [10]:
def cleanhtml(raw_html):
  # 1.Source code in python language is hard to understand, so replace all the <code> tag first
  cleantext_1 = re.findall(r'(?<=\<code>)(.*?)(?=<\/code>)', raw_html.replace('\n', '_**_'))
  cleantext_1 = [x.replace('_**_', '\n') for x in cleantext_1]
  # 2. replace html tags
  # <p>
  tag_re = re.compile('<.*?>')
  cleantext_2 = [re.sub(tag_re, '', x) for x in cleantext_1]
  return cleantext_2

In [11]:
# apply the function, cleanhtml to the question and body text
bert_src.loc[:, 'q_prep_text'] = bert_src['body'].apply(cleanhtml)

In [12]:
bert_src = bert_src.reset_index(drop=True)

In [13]:
bert_src_ex = bert_src[['q_id', 'q_prep_text']].apply(pd.Series.explode)

In [14]:
bert_src_ex.dropna(inplace=True)

In [15]:
bert_src = pd.merge(bert_src_ex, bert_src[['q_id', 'q_creationdate']], how ='left', left_on = 'q_id', right_on = 'q_id')

In [16]:
bert_src['q_year'] = pd.to_datetime(bert_src['q_creationdate']).dt.year
bert_src['q_month'] = pd.to_datetime(bert_src['q_creationdate']).dt.month
bert_src['q_day'] = pd.to_datetime(bert_src['q_creationdate']).dt.day

In [17]:
# PREPROCESSING FOR CODE SCRIPT
def preprocess_script(script):
    new_script = deque()
    old_script = script.split('\n')
    for line in old_script:
        if line.lstrip().startswith('#'): # 주석으로 시작되는 행 skip
            continue
        line = line.rstrip()
        if '#' in line:
            line = line[:line.index('#')] # 주석 전까지 코드만 저장
        line = line.replace('\n','') # 개행 문자를 모두 삭제함
        line = line.replace('    ','\t') # 공백 4칸을 tab으로 변환
        
        if line == '': # 전처리 후 빈 라인은 skip
            continue
        
        new_script.append(line)

        
    new_script = '\n'.join(new_script) # 개행 문자로 합침
    #이렇게 할지 ..... 고민을 해봐야 할것같음... 전처리......
    new_script = re.sub('("""[\w\W]*?""")', '', new_script)
    new_script = re.sub("('''[\w\W]*?''')", '', new_script)
    new_script = re.sub('/^(http?|https?):\/\/([a-z0-9-]+\.)+[a-z0-9]{2,4}.*$/', '', new_script)
    
    return new_script


In [18]:
bert_src['q_prep_text_non'] = bert_src['q_prep_text'].apply(preprocess_script)

In [19]:
# bert_src.loc[bert_src['q_prep_text_non']!= bert_src['q_prep_text'], ['q_prep_text', 'q_prep_text_non']].head(100).to_csv('chk_preprocess.csv')

In [20]:
bert_src.iloc[51:54, :]

,q_id,q_prep_text,q_creationdate,q_year,q_month,q_day,q_prep_text_non
51,70163035,mask,2021-11-30 00:54:03.373,2021,11,30,mask
52,70163035,bool,2021-11-30 00:54:03.373,2021,11,30,bool
53,70163035,x,2021-11-30 00:54:03.373,2021,11,30,x


In [21]:
# CodeBERT 모델과 토크나이저 로드
model_name = "microsoft/codebert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [22]:
# 임베딩 함수 정의
def embed_text(text):
    # 텍스트를 토크나이저를 사용해 인코딩
    encoded_input = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    
    # 모델 추론 시 기울기를 계산하지 않도록 설정 > 기울기를 계산하여 파라미터를 업데이트 하지 않는다고?
    with torch.no_grad():
        # 인코딩된 입력을 모델에 전달하여 출력 생성(key:value로 해서 model output에 전달)
        model_output = model(**encoded_input)
    
    # 모델의 출력에서 [CLS] 토큰의 임베딩을 반환
    # 입력시퀀스의 각 토큰의 마지막 은닉상태를 가져오기 위해 last_hidden_state에 접근
    # [batch_size, sequence_length, hidden_size]의 텐서
    # 중간 파라미터를 0으로 두어 첫번째 토큰인 cls를 가져옴
    # 그래서 결과는 [batch_size, hidden_size가 됨]
    return model_output.last_hidden_state[:, 0, :]

In [23]:
years = [2021,2022, 2023]
months = [x for x in range(1, 13)]
ym = list(product(years, months))

In [24]:
ym[10:-1]

[(2021, 11),
 (2021, 12),
 (2022, 1),
 (2022, 2),
 (2022, 3),
 (2022, 4),
 (2022, 5),
 (2022, 6),
 (2022, 7),
 (2022, 8),
 (2022, 9),
 (2022, 10),
 (2022, 11),
 (2022, 12),
 (2023, 1),
 (2023, 2),
 (2023, 3),
 (2023, 4),
 (2023, 5),
 (2023, 6),
 (2023, 7),
 (2023, 8),
 (2023, 9),
 (2023, 10),
 (2023, 11)]

In [25]:
bert_src

,q_id,q_prep_text,q_creationdate,q_year,q_month,q_day,q_prep_text_non
0,70162722,def text_to_html(text):\n &quot;&quot;&quot...,2021-11-30 00:00:11.787,2021,11,30,def text_to_html(text):\n\t&quot;&quot;&quot;\...
1,70162722,class TestDebtModules(unittest.TestCase):\n ...,2021-11-30 00:00:11.787,2021,11,30,class TestDebtModules(unittest.TestCase):\n\td...
2,70162722,Expected :&lt;p&gt;&lt;br&gt; THIS&lt;b...,2021-11-30 00:00:11.787,2021,11,30,Expected :&lt;p&gt;&lt;br&gt;\t\tTHIS&lt;br&gt...
3,70162741,vid sid pid url \n1 A A1 pag...,2021-11-30 00:03:41.067,2021,11,30,vid sid pid\t url\n1 A\tA1 page\t A...
4,70162741,"lst = ['AB','EF']",2021-11-30 00:03:41.067,2021,11,30,"lst = ['AB','EF']"
...,...,...,...,...,...,...,...
1197066,77581609,"self.execute(record.last_modified, record.crea...",2023-11-30 20:47:13.247,2023,11,30,"self.execute(record.last_modified, record.crea..."
1197067,77581609,"my_list = [&quot;last_modified&quot;, &quot;cr...",2023-11-30 20:47:13.247,2023,11,30,"my_list = [&quot;last_modified&quot;, &quot;cr..."
1197068,77581609,"my_list = [&quot;last_modified&quot;, &quot;cr...",2023-11-30 20:47:13.247,2023,11,30,"my_list = [&quot;last_modified&quot;, &quot;cr..."
1197069,77581621,==============================================...,2023-11-30 20:49:03.050,2023,11,30,==============================================...


In [39]:
q_id_df  = pd.DataFrame()

for year, month in ym[10:-1]:
    startend = calendar.monthrange(year, month)
    

    
    cond1 = (bert_src['q_year'] ==year)
    cond2 = (bert_src['q_month'] ==month)

    q_id_list = bert_src.loc[cond1&cond2, 'q_id'].unique()
    num_of_sample = int(len(q_id_list)*0.1)
    print(year, month, len(q_id_list))
    q_id_df = pd.concat([q_id_df, pd.DataFrame(q_id_list).sample(num_of_sample,replace=False)], axis = 0)



2021 11 716
2021 12 17954
2022 1 18748
2022 2 18051
2022 3 19517
2022 4 18661
2022 5 18293
2022 6 17708
2022 7 17926
2022 8 17763
2022 9 16375
2022 10 17390
2022 11 17861
2022 12 18196
2023 1 17134
2023 2 15141
2023 3 15045
2023 4 12160
2023 5 11463
2023 6 10817
2023 7 10787
2023 8 9984
2023 9 8407
2023 10 8905
2023 11 9720


In [40]:
q_id_df.columns  = ['q_id']

In [50]:
q_id_df['q_id'].values

array([70168326, 70163975, 70170522, ..., 77544653, 77414212, 77577334])

In [52]:
embed_text_list = []
for year, month in ym[10:-1]:
    startend = calendar.monthrange(year, month)
    day = startend[1]

    print(year, month, day) 
    cond1 = (bert_src['q_year'] ==year)
    cond2 = (bert_src['q_month'] ==month)
    # cond3 = (bert_src['q_day'] ==day)
    cond4 = (bert_src['q_id'].isin(q_id_df['q_id'].values))
    src = bert_src.loc[cond1&cond2&cond4, 'q_prep_text_non'].tolist()
    for i in tqdm(range(len(src))) : 
        embed_text_list.append(embed_text(src[i]))

2021 11 30


  0%|          | 0/172 [00:00<?, ?it/s]

 72%|███████▏  | 124/172 [00:08<00:02, 18.00it/s]

In [ ]:
data = pd.DataFrame()
for year, month in ym[10:-1]:
    startend = calendar.monthrange(year, month)
    day = startend[1]

    print(year, month, day) 
    cond1 = (bert_src['q_year'] ==year)
    cond2 = (bert_src['q_month'] ==month)
    # cond3 = (bert_src['q_day'] ==day)
    cond4 = (bert_src['q_id'].isin(q_id_df['q_id'].values))
    data = pd.concat([data, bert_src.loc[cond1&cond2&cond4, :]], axis = 0)


In [ ]:
# 파일 불러오기
with open('../../data/reserved_list.pkl', 'rb') as f:
    reserved_list = pickle.load(f)


In [ ]:
src = data['q_prep_text'].to_list()

In [ ]:
# CodeBERT 모델과 토크나이저 로드
model_name = "microsoft/codebert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [ ]:
# 모든 텍스트에 대한 임베딩 계산
embeddings = torch.cat(embed_text_list, dim=0).numpy()  # 텐서로 연결 후 numpy 배열로 변환

In [ ]:
embeddings.shape

In [ ]:
umap_model = UMAP(n_neighbors=10, n_components=5, min_dist=0.0, metric='cosine', random_state=42)

In [ ]:
hdbscan_model = HDBSCAN(min_cluster_size=20, metric='euclidean', cluster_selection_method='eom', prediction_data=True)
cluster_model = KMeans(n_clusters=10)

In [ ]:
# KeyBERT
keybert_model = KeyBERTInspired()

# Part-of-Speech
# pos_model = PartOfSpeech("en_core_web_sm")

# MMR
mmr_model = MaximalMarginalRelevance(diversity=0.3)

# GPT-3.5
prompt = """
I have a topic that contains the documents: [REPRESENTATIVE_DOCUMENT]
The topic is described by the following keywords: [KEYWORDS]

Based on the information above, extract a short but highly descriptive topic label of at most 5 words. Make sure it is in the following format:
topic: <topic label>
"""
client = openai.OpenAI(api_key="sk-proj-Mm0FIOmjCUgXc2T40TarT3BlbkFJ7YO5SE9kJokyhFrRxEiB")
openai_model = OpenAI(client, model="gpt-3.5-turbo", exponential_backoff=True, chat=True, prompt=prompt)

# All representation models
representation_model = {
    "KeyBERT": keybert_model,
    "OpenAI": openai_model,  # Uncomment if you will use OpenAI
    "MMR": mmr_model
    # ,
    # "POS": pos_model
}

In [ ]:
vectorizer_model = CountVectorizer(stop_words=reserved_list, min_df=2, ngram_range=(1, 2))

In [ ]:
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)


In [ ]:
# BERTopic 모델 초기화 및 훈련
topic_model = BERTopic( embedding_model=model,
                        umap_model=umap_model,
                        # hdbscan_model=hdbscan_model,
                        hdbscan_model=cluster_model,
                        ctfidf_model=ctfidf_model,
                        vectorizer_model = vectorizer_model,
                        representation_model=representation_model)  # 임베딩 모델 사용을 비활성화
topics, probabilities = topic_model.fit_transform(src, embeddings)


In [ ]:
# 결과 출력
print(topic_model.get_topic_info())  # 토픽 정보 출력

In [ ]:
topic_model.get_topic_info()

In [ ]:
topic_model.get_topic(0)

In [ ]:
topic_model.get_topic(1)

In [ ]:
topic_model.get_topic(6, full=True)

In [ ]:
# # Label the topics yourself
# topic_model.set_topic_labels({1: "Space Travel", 7: "Religion"})

# # or use one of the other topic representations, like KeyBERTInspired
# keybert_topic_labels = {topic: " | ".join(list(zip(*values))[0][:3]) for topic, values in topic_model.topic_aspects_["KeyBERT"].items()}
# topic_model.set_topic_labels(keybert_topic_labels)

# # or ChatGPT's labels
# chatgpt_topic_labels = {topic: " | ".join(list(zip(*values))[0]) for topic, values in topic_model.topic_aspects_["OpenAI"].items()}
# chatgpt_topic_labels[-1] = "Outlier Topic"
# topic_model.set_topic_labels(chatgpt_topic_labels)

In [ ]:
# topic_distr, _ = topic_model.approximate_distribution(data, window=8, stride=4)

In [ ]:
# pip install nbformat>=4.2.0

In [ ]:
# Visualize the topic-document distribution for a single document
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_hierarchy()

In [ ]:
reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)

In [ ]:
topic_model.visualize_documents(src, embeddings=embeddings)

In [ ]:
topic_model.visualize_documents(src, reduced_embeddings=reduced_embeddings)

In [ ]:
topa_df = topic_model.get_document_info(src)

In [ ]:
topa_df

In [ ]:
topa_org_df = data.copy()
topa_org_df.reset_index(drop=True, inplace=True)
topa_df.reset_index(drop=True, inplace=True)

In [ ]:
topa_df

In [ ]:
tot_topa_df = pd.concat([topa_org_df, topa_df], axis = 1)

In [ ]:
tot_topa_df

In [ ]:
tot_topa_df[(tot_topa_df['Topic'] ==0)&(tot_topa_df['Representative_document'] ==True)]

In [ ]:
data

In [ ]:
sample_dropnull_df = sample_df[['q_id', 'q_creationdate', 'a_id']].groupby(['q_id', 'q_creationdate']).head(1)
sample_dropnull_df = sample_dropnull_df[['q_id', 'a_id']]

In [ ]:
tot_topa_df = pd.merge(tot_topa_df, sample_dropnull_df, how='left', left_on = 'q_id', right_on = 'q_id')

In [ ]:
tot_topa_df[tot_topa_df['q_id'] == 77394187]

In [ ]:
tot_topa_df

In [ ]:
data

In [ ]:
tot_topa_df['date'] = pd.to_datetime(tot_topa_df['q_creationdate']).dt.date

In [ ]:
tot_topa_df

In [ ]:
tot_topa_not_answered_df = tot_topa_df[tot_topa_df['a_id'].isna()]
tot_topa_answered_df = tot_topa_df[tot_topa_df['a_id'].isna()==False]

In [ ]:
tot_topa_not_answered_rate = tot_topa_not_answered_df.groupby(['date', 'Topic']).count()[['q_id']].reset_index()
tot_topa_answered_rate = tot_topa_answered_df.groupby(['date', 'Topic']).count()[['q_id']].reset_index()

In [ ]:
tot_topa_not_answered_sum = tot_topa_not_answered_rate.groupby(['date']).sum()[['q_id']].reset_index()
tot_topa_answered_sum = tot_topa_answered_rate.groupby(['date']).sum()[['q_id']].reset_index()

In [ ]:
tot_topa_not_answered_rate = pd.merge(tot_topa_not_answered_rate, tot_topa_not_answered_sum, on = 'date')
tot_topa_answered_rate = pd.merge(tot_topa_answered_rate, tot_topa_answered_sum, on = 'date')

In [ ]:
tot_topa_not_answered_rate

In [ ]:
# topic_per_rate = tot_topa_not_answered_rate.rename({'q_id_x': 'cnt', 'q_id_y':'tot_cnt'})

In [ ]:
tot_topa_not_answered_rate['rate'] = tot_topa_not_answered_rate['q_id_x']/tot_topa_not_answered_rate['q_id_y']*100
tot_topa_answered_rate['rate'] = tot_topa_answered_rate['q_id_x']/tot_topa_answered_rate['q_id_y']*100


In [ ]:
tot_topa_not_answered_rate

In [ ]:
tot_topa_not_answered_rate_pivot = pd.pivot_table(tot_topa_not_answered_rate, index = 'date', columns = 'Topic',values = 'rate').reset_index()
tot_topa_answered_rate_pivot = pd.pivot_table(tot_topa_answered_rate, index = 'date', columns = 'Topic',values = 'rate').reset_index()

In [ ]:
tot_topa_not_answered_pivot = pd.pivot_table(tot_topa_not_answered_rate, index = 'date', columns = 'Topic',values = 'q_id_x').reset_index()
tot_topa_answered_pivot = pd.pivot_table(tot_topa_answered_rate, index = 'date', columns = 'Topic',values = 'q_id_x').reset_index()

In [ ]:
tot_topa_not_answered_rate_pivot.fillna(0, inplace=True)
tot_topa_answered_rate_pivot.fillna(0, inplace=True)

In [ ]:
tot_topa_not_answered_pivot.fillna(0, inplace=True)
tot_topa_answered_pivot.fillna(0, inplace=True)

In [ ]:
tot_topa_not_answered_rate_pivot

In [ ]:
tot_topa_not_answered_rate_pivot['sdate'] = tot_topa_not_answered_rate_pivot['date'].astype('str')
tot_topa_answered_rate_pivot['sdate'] = tot_topa_answered_rate_pivot['date'].astype('str')

In [ ]:
tot_topa_not_answered_pivot.to_csv('./chk_not_answered.csv')
tot_topa_answered_pivot.to_csv('./chk_answered.csv')

In [ ]:
### Problem 2b: Reformat the data and generate the stacked bar plot above.


fig, ax = plt.subplots(figsize = (10, 5))

# ax.bar(topic_per_pivot['date'], topic_per_pivot[0], width = 0.5, color='blue', label = 'Party A')
ax.bar(tot_topa_answered_rate_pivot['sdate'], tot_topa_not_answered_rate_pivot[0], color='red', label = 'Party A')
ax.bar(tot_topa_answered_rate_pivot['sdate'], tot_topa_not_answered_rate_pivot[1], bottom = tot_topa_not_answered_rate_pivot[0], color = 'orange', label = 'Party B')
ax.bar(tot_topa_answered_rate_pivot['sdate'], tot_topa_not_answered_rate_pivot[2], bottom = tot_topa_not_answered_rate_pivot[1] + tot_topa_not_answered_rate_pivot[0], color = 'green', label = 'Party B')
ax.bar(tot_topa_answered_rate_pivot['sdate'], tot_topa_not_answered_rate_pivot[3], bottom = tot_topa_not_answered_rate_pivot[2] + tot_topa_not_answered_rate_pivot[1] + tot_topa_not_answered_rate_pivot[0], color = 'yellow', label = 'Party B')
ax.bar(tot_topa_answered_rate_pivot['sdate'], tot_topa_not_answered_rate_pivot[4], bottom = tot_topa_not_answered_rate_pivot[3] + tot_topa_not_answered_rate_pivot[2] + tot_topa_not_answered_rate_pivot[1] + tot_topa_not_answered_rate_pivot[0], color = 'blue', label = 'Party B')
ax.bar(tot_topa_answered_rate_pivot['sdate'], tot_topa_not_answered_rate_pivot[5], bottom = tot_topa_not_answered_rate_pivot[4] + tot_topa_not_answered_rate_pivot[3] + tot_topa_not_answered_rate_pivot[2] + tot_topa_not_answered_rate_pivot[1] + tot_topa_not_answered_rate_pivot[0], color = 'purple', label = 'Party B')


# ax.set_xlabel('Congress', fontsize = 5, labelpad = 20)
# ax.set_xlabel('Congress', fontsize = 5)
# ax.set_ylabel('Number of Seats', fontsize = 5, labelpad = 20)
# ax.set_title('Partisan Composition of Congress', fontsize = 17.5, pad = 20)

# ax.tick_params(axis = 'both', labelsize = 5)

# ax.legend(title = "Political Party",
#           title_fontsize = 15, fontsize = 5,
#           bbox_to_anchor = (1, 0.55))

plt.xticks(rotation = 45)
plt.show();

In [ ]:
### Problem 2b: Reformat the data and generate the stacked bar plot above.


fig, ax = plt.subplots(figsize = (10, 5))

# ax.bar(topic_per_pivot['date'], topic_per_pivot[0], width = 0.5, color='blue', label = 'Party A')
ax.bar(tot_topa_answered_rate_pivot['sdate'], tot_topa_answered_rate_pivot[0], color='red', label = 'Party A')
ax.bar(tot_topa_answered_rate_pivot['sdate'], tot_topa_answered_rate_pivot[1], bottom = tot_topa_answered_rate_pivot[0], color = 'orange', label = 'Party B')
ax.bar(tot_topa_answered_rate_pivot['sdate'], tot_topa_answered_rate_pivot[2], bottom = tot_topa_answered_rate_pivot[1] + tot_topa_answered_rate_pivot[0], color = 'green', label = 'Party B')
ax.bar(tot_topa_answered_rate_pivot['sdate'], tot_topa_answered_rate_pivot[3], bottom = tot_topa_answered_rate_pivot[2] + tot_topa_answered_rate_pivot[1] + tot_topa_answered_rate_pivot[0], color = 'yellow', label = 'Party B')
ax.bar(tot_topa_answered_rate_pivot['sdate'], tot_topa_answered_rate_pivot[4], bottom = tot_topa_answered_rate_pivot[3] + tot_topa_answered_rate_pivot[2] + tot_topa_answered_rate_pivot[1] + tot_topa_answered_rate_pivot[0], color = 'blue', label = 'Party B')
ax.bar(tot_topa_answered_rate_pivot['sdate'], tot_topa_answered_rate_pivot[5], bottom = tot_topa_answered_rate_pivot[4] + tot_topa_answered_rate_pivot[3] + tot_topa_answered_rate_pivot[2] + tot_topa_answered_rate_pivot[1] + tot_topa_answered_rate_pivot[0], color = 'purple', label = 'Party B')


# ax.set_xlabel('Congress', fontsize = 5, labelpad = 20)
# ax.set_xlabel('Congress', fontsize = 5)
# ax.set_ylabel('Number of Seats', fontsize = 5, labelpad = 20)
# ax.set_title('Partisan Composition of Congress', fontsize = 17.5, pad = 20)

# ax.tick_params(axis = 'both', labelsize = 5)

# ax.legend(title = "Political Party",
#           title_fontsize = 15, fontsize = 5,
#           bbox_to_anchor = (1, 0.55))

plt.xticks(rotation = 45)
plt.show();

In [ ]:
idx = np.arange(tot_topa_answered_rate_pivot.shape[0])
w = 0.15

plt.figure(figsize = (10, 5))
plt.title('pctchg in topic 1')
plt.xlabel('month')
plt.ylabel('pct')
plt.plot(tot_topa_answered_rate_pivot[0], label = 'answered')
plt.plot(tot_topa_not_answered_rate_pivot[0], label = 'not answered')
plt.xticks(idx, tot_topa_answered_rate_pivot['sdate'], rotation = 30)
plt.legend(ncol = 1)
plt.show()

In [ ]:
idx = np.arange(tot_topa_answered_rate_pivot.shape[0])
w = 0.15

plt.figure(figsize = (10, 5))
plt.title('pctchg in topic 2')
plt.xlabel('month')
plt.ylabel('pct')
plt.plot(tot_topa_answered_rate_pivot[1], label = 'answered')
plt.plot(tot_topa_not_answered_rate_pivot[1], label = 'not answered')
plt.xticks(idx, tot_topa_answered_rate_pivot['sdate'], rotation = 30)
plt.legend(ncol = 1)
plt.show()

In [ ]:
idx = np.arange(tot_topa_answered_rate_pivot.shape[0])
w = 0.15

plt.figure(figsize = (10, 5))
plt.title('pctchg in topic 3')
plt.xlabel('month')
plt.ylabel('pct')
plt.plot(tot_topa_answered_rate_pivot[2], label = 'answered')
plt.plot(tot_topa_not_answered_rate_pivot[2], label = 'not answered')
plt.xticks(idx, tot_topa_answered_rate_pivot['sdate'], rotation = 30)
plt.legend(ncol = 1)
plt.show()

In [ ]:
idx = np.arange(tot_topa_answered_rate_pivot.shape[0])
w = 0.15

plt.figure(figsize = (10, 5))
plt.title('pctchg in topic 4')
plt.xlabel('month')
plt.ylabel('pct')
plt.plot(tot_topa_answered_rate_pivot[3], label = 'answered')
plt.plot(tot_topa_not_answered_rate_pivot[3], label = 'not answered')
plt.xticks(idx, tot_topa_answered_rate_pivot['sdate'], rotation = 30)
plt.legend(ncol = 1)
plt.show()

In [ ]:
idx = np.arange(tot_topa_answered_rate_pivot.shape[0])
w = 0.15

plt.figure(figsize = (10, 5))
plt.title('pctchg in topic 5')
plt.xlabel('month')
plt.ylabel('pct')
plt.plot(tot_topa_answered_rate_pivot[4], label = 'answered')
plt.plot(tot_topa_not_answered_rate_pivot[4], label = 'not answered')
plt.xticks(idx, tot_topa_answered_rate_pivot['sdate'], rotation = 30)
plt.legend(ncol = 1)
plt.show()

In [ ]:
tot_topa_answered_rate_pivot

In [ ]:
tot_topa_answered_rate_pivot